# Exploratory Data Analysis in pySpark II 

In [ ]:
# set up 
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

from pyspark.sql.types import *
from pyspark.sql.functions import *

sc = SparkContext(master = "local" \
                , appName = "Exploratory Data Analysis") 

# Create a spark session 
spark = SparkSession(sc)
sqlContext = SQLContext(spark)

In [ ]:
# Files we will be working with
path = "/home/danae/Documents/pySparkTraining/files/"

df = spark.read.parquet(path + "olimpic_medals.parquet")
df.show(5)

## Univariate Analysis

You have several options for visualizing and describing univariate data. Such as frequency Distribution Tables, bar Charts, histograms, frequency Polygons, pie Charts, etc.

In [ ]:
num_cols = ['age','height', 'weigth']
df.select(num_cols).describe().show()

You may find out that the default function in PySpark does not include the quartiles. The following function will help you to get the same results in Pandas

In [ ]:
import numpy as np 
import pandas as pd 

def describe_pd(df_in, columns, deciles=False):
    '''
    Function to union the basic stats results and deciles
    :param df_in: the input dataframe
    :param columns: the cloumn name list of the numerical variable
    :param deciles: the deciles output

    :return : the numerical describe info. of the input dataframe

    :author: Ming Chen and Wenqiang Feng
    :email:  von198@gmail.com
    '''

    if deciles:
        percentiles = np.array(range(0, 110, 10))
    else:
        percentiles = [25, 50, 75]

    percs = np.transpose([np.percentile(df_in.select(x).collect(), percentiles) for x in columns])
    percs = pd.DataFrame(percs, columns=columns)
    percs['summary'] = [str(p) + '%' for p in percentiles]

    spark_describe = df_in.describe().toPandas()
    new_df = pd.concat([spark_describe, percs],ignore_index=True)
    new_df = new_df.round(2)
    return new_df[['summary'] + columns]

In [ ]:
describe_pd(df, num_cols)

Sometimes, because of the confidential data issues, you can not deliver the real data and your clients may ask more statistical results, such as deciles. You can apply the follwing function to achieve it.

In [ ]:
describe_pd(df, num_cols, deciles=True)

## Skewness and Kurtosis

In [ ]:
from pyspark.sql.functions import col, skewness, kurtosis
df.select(skewness('age'),kurtosis('age')).show()

## Frequency table

In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.functions import rank,sum,col
from pyspark.sql import Window

window = Window.rowsBetween(Window.unboundedPreceding,Window.unboundedFollowing)
# withColumn('Percent %',F.format_string("%5.0f%%\n",col('Credit_num')*100/col('total'))).\
tab = df.select(['gender','weigth']).\
   groupBy('gender').\
   agg(F.count('weigth').alias('n'),
       F.mean('weigth').alias('avg'),
       F.min('weigth').alias('min'),
       F.max('weigth').alias('max')).\
   withColumn('total', sum(col('n')).over(window)).\
   withColumn('Percent',col('n')*100/col('total')).\
   drop(col('total'))

In [ ]:
tab.show()

## Multivariate Analysis

### Correlation matrix

In [ ]:
from pyspark.mllib.stat import Statistics
import pandas as pd

corr_data = df.select(num_cols)

col_names = corr_data.columns
features = corr_data.rdd.map(lambda row: row[0:])
corr_mat = Statistics.corr(features, method="pearson")
corr_df = pd.DataFrame(corr_mat)
corr_df.index, corr_df.columns = col_names, col_names

print(corr_df.to_string())

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="ticks")

#df = sns.load_dataset("iris")
df_plot = df['age','height', 'weigth', 'gender']
df_plot.show(5)

In [ ]:
sns.pairplot(df_plot.toPandas(), hue = "gender")
plt.show()

### Cross table

In [ ]:
df.stat.crosstab('season', 'gender').show()

In [ ]:
sc.stop() # close the spark session